In [1]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tuankietnguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/tuankietnguyen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [2]:
import pandas as pd

df_pd = pd.read_csv("../Data/csv/reuters_all.csv")
# df_pd = df_pd.sample(n=5, random_state=42)
print("📊 Tổng số bài viết:", len(df_pd))
print("📌 Các cột:", df_pd.columns.tolist())
print("\n📝 Xem Thông tin:")
print(df_pd.info())
df_pd = df_pd.dropna(subset=['body'])
df_pd = df_pd.reset_index(drop=True)
print(df_pd.info())

📊 Tổng số bài viết: 21578
📌 Các cột: ['id', 'title', 'body', 'topics']

📝 Xem Thông tin:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21578 entries, 0 to 21577
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      21578 non-null  int64 
 1   title   20841 non-null  object
 2   body    19043 non-null  object
 3   topics  11367 non-null  object
dtypes: int64(1), object(3)
memory usage: 674.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19043 entries, 0 to 19042
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      19043 non-null  int64 
 1   title   19043 non-null  object
 2   body    19043 non-null  object
 3   topics  10377 non-null  object
dtypes: int64(1), object(3)
memory usage: 595.2+ KB
None


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col
import os

os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SemanticsBasedDistributedDocumentClustering_SparkNLP") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .config("spark.executorEnv.NLTK_DATA", "/Users/tuankietnguyen/nltk_data") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
df = spark.createDataFrame(df_pd)

df = df.withColumn("body", concat_ws(" ", col("title"), col("body")))
df.show(5)

25/07/03 23:15:50 WARN Utils: Your hostname, MBP-of-Yeun.local resolves to a loopback address: 127.0.0.1; using 192.168.50.97 instead (on interface en0)
25/07/03 23:15:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/tuankietnguyen/.ivy2/cache
The jars for the packages stored in: /Users/tuankietnguyen/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c92c572f-1af1-4c1f-909b-2abb71e06db3;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in ce

+---+--------------------+--------------------+--------------------+
| id|               title|                body|              topics|
+---+--------------------+--------------------+--------------------+
|  1|  BAHIA COCOA REVIEW|BAHIA COCOA REVIE...|               cocoa|
|  2|STANDARD OIL <SRD...|STANDARD OIL <SRD...|                 NaN|
|  3|TEXAS COMMERCE BA...|TEXAS COMMERCE BA...|                 NaN|
|  4|TALKING POINT/BAN...|TALKING POINT/BAN...|                 NaN|
|  5|NATIONAL AVERAGE ...|NATIONAL AVERAGE ...|grain;wheat;corn;...|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows



In [4]:
from pyspark.ml import Pipeline
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, LemmatizerModel, PerceptronModel
from sparknlp.base import DocumentAssembler, Finisher

document_assembler = DocumentAssembler().setInputCol("body").setOutputCol("document")

tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("raw_tokens")

stopwords_cleaner = StopWordsCleaner.pretrained("stopwords_en", "en") \
    .setInputCols(["raw_tokens"]) \
    .setOutputCol("clean_tokens")

normalizer = Normalizer() \
    .setInputCols(["clean_tokens"]) \
    .setOutputCol("normalized_tokens") \
    .setLowercase(True) \
    .setCleanupPatterns(["[^A-Za-z]"])

pos_tagger = PerceptronModel.pretrained("pos_anc", "en") \
    .setInputCols(["document", "normalized_tokens"]) \
    .setOutputCol("pos_tags")
    
lemmatizer = LemmatizerModel.pretrained("lemma_antbnc", "en") \
    .setInputCols(["normalized_tokens"]) \
    .setOutputCol("lemmas")

finisher = Finisher() \
    .setInputCols(["lemmas", "pos_tags"]) \
    .setOutputCols(["fin_tokens", "final_pos"])

nlp_pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    stopwords_cleaner,
    normalizer,
    pos_tagger,
    lemmatizer,
    finisher
])

print("⏳ Bắt đầu tiền xử lý với Spark NLP...")
df_processed = nlp_pipeline.fit(df).transform(df)
df_processed.cache()
print("✅ Xong bước NLP")

df_processed.select("fin_tokens", "final_pos").show(5, truncate=False)

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[ / ]stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
Download done! Loading the resource.
[OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ]pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ / ]Download done! Loading the resource.


[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ / ]Download done! Loading the resource.


OpenJDK 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
OpenJDK 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=30283Kb max_used=30468Kb free=100788Kb
 bounds [0x0000000106184000, 0x0000000107f74000, 0x000000010e184000]
 total_blobs=11169 nmethods=10214 adapters=867
 compilation: disabled (not enough contiguous free space left)
[OK!]
⏳ Bắt đầu tiền xử lý với Spark NLP...
✅ Xong bước NLP


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType

@udf(ArrayType(StringType()))
def wsd_udf(tokens, pos_tags):
    from nltk.corpus import wordnet as wn
    from nltk.wsd import lesk
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wn.ADJ
        elif tag.startswith('V'):
            return wn.VERB
        elif tag.startswith('N'):
            return wn.NOUN
        elif tag.startswith('R'):
            return wn.ADV
        else:
            return None
    result = []
    for word, tag in zip(tokens, pos_tags):
        pos = get_wordnet_pos(tag)
        syn = lesk(tokens, word, pos)
        if syn :
            result.append(syn.name())
    return result

df_wsd = (
    df_processed
    .withColumn("senses", wsd_udf(col("fin_tokens"), col("final_pos")))
    .drop("fin_tokens", "final_pos") 
)
print(df_wsd.columns)
df_wsd.select("senses").show(5, truncate=False)

['id', 'title', 'body', 'topics', 'senses']


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
@udf(ArrayType(StringType()))
def expand_syn_hyper(sids):
    from nltk.corpus import wordnet as wn
    if not sids:
        return []

    out = set()
    for sid in sids:
        if not sid:
            continue
        if "." in sid:
            try:
                syn = wn.synset(sid)
                out.update(l.name() for l in syn.lemmas())
                continue        
            except Exception:
                pass             
        out.add(sid)

    return list(out)

df_expanded = (
    df_wsd.select("id", "topics", "senses")
    .withColumn("sem_tokens", expand_syn_hyper(col("senses")))
    .drop("senses") 
)
print(df_expanded.columns)
df_expanded.select("sem_tokens").show(5, truncate=False)



['id', 'topics', 'sem_tokens']


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:

def synset2concepts(sid, max_depth=1):
    from nltk.corpus import wordnet as wn
    try:
        syn = wn.synset(sid)
    except:
        return ["unknown"]
    
    paths = syn.hypernym_paths()
    if not paths:
        return [syn.lexname()]
    
    path = paths[0]
    concepts = []

    for i in range(0, min(len(path), max_depth + 1)):
        lemma = path[-(i + 1)].lemma_names()[0]
        if lemma not in concepts:
            concepts.append(lemma)
    return concepts

from pyspark.sql.functions import col, explode
unique_synsets = (
    df_wsd
    .select(explode(col("senses")).alias("sid"))
    .where(col("sid").isNotNull())
    .distinct()
    .toPandas()["sid"]
    .tolist()
)

concept_map = {}
for sid in unique_synsets:
    key = sid.split('.')[0] 
    concepts = synset2concepts(sid)
    concept_map[key] = concepts
    
b_concept_map = spark.sparkContext.broadcast(concept_map)

@udf(ArrayType(StringType()))
def map_to_concepts(tokens):
    out = []
    for t in tokens:
        concepts = b_concept_map.value.get(t, [t]) 
        out.extend(concepts)
    return list(dict.fromkeys(out)) 

df_semantic = df_expanded.withColumn(
    "final_tokens", map_to_concepts(col("sem_tokens"))
).drop("sem_tokens")
print(df_semantic.columns)
df_semantic.select("final_tokens").show(5, truncate=False)


['id', 'topics', 'final_tokens']
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
from pyspark.ml.clustering import KMeans, BisectingKMeans
from pyspark.sql.functions import col, split, explode, log2, countDistinct
def calculate_purity_entropy(df, label_col="topic", pred_col="prediction"):
    df_unique = df.select("id", pred_col, label_col).dropDuplicates(["id"])
    total = df_unique.select("id").distinct().count()
    if total == 0:
        return None, None
    counts = df.groupBy(pred_col, label_col).count()
    cluster_sizes = df.select(pred_col, "id").dropDuplicates(["id", pred_col]) \
    .groupBy(pred_col) \
    .agg(countDistinct("id").alias("cluster_total"))
    joined = counts.join(cluster_sizes, pred_col)

    max_counts = joined.groupBy(pred_col).agg({"count": "max"}).withColumnRenamed("max(count)", "max_count")
    total_purity = max_counts.agg({"max_count": "sum"}).collect()[0][0]
    purity = total_purity / total

    entropy_df = joined.withColumn("p", col("count") / col("cluster_total"))
    entropy_df = entropy_df.withColumn("p_log_p", -col("p") * log2(col("p")))
    entropy = entropy_df.groupBy(pred_col).agg({"p_log_p": "sum"}).agg({"sum(p_log_p)": "avg"}).collect()[0][0]

    return purity, entropy
results = []

variant = "Pos + worknet"
from pyspark.ml.feature import IDF, HashingTF
numFeatures_stage = [9820]
for i in numFeatures_stage:
    print(f"\n============================== numFeatures={i} ==============================")
    hashingTF = HashingTF(inputCol="final_tokens", outputCol="rawFeatures", numFeatures=i)
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    df_tf = hashingTF.transform(df_semantic)
    idf_model = idf.fit(df_tf)
    df_features = idf_model.transform(df_tf)
    df_features = df_features.select("id", "topics", "features")
    print(df_features.columns)
    # df_features.select("features").show(5, truncate=False)


    k_stage = [5,6,7,8]
    for k in k_stage:
        print(f"\n============================== k={k} ==============================")
        
        print(f"\nKMeans")
        km = KMeans(k=k, featuresCol="features", predictionCol="prediction", seed=42)
        pred_km = km.fit(df_features).transform(df_features)
        
        pred_km_filtered = pred_km.filter(col("topics").isNotNull() & (col("topics") != "NaN"))
        pred_km_filtered = pred_km_filtered.withColumn("topics_array", split(col("topics"), ";"))
        pred_km_exploded = pred_km_filtered.withColumn("topic", explode(col("topics_array")))
        
        purity_km, entropy_km = calculate_purity_entropy(pred_km_exploded)
        if purity_km is not None:
            print(f"Purity: {purity_km:.4f}")
            print(f"Entropy: {entropy_km:.4f}")
            results.append({
                "k": k,
                "method": "KMeans",
                "variant": variant,
                "purity": purity_km,
                "entropy": entropy_km
            })


        print(f"\nBisectingKMeans")
        bkm = BisectingKMeans(k=k, featuresCol="features", predictionCol="prediction", seed=42)
        pred_bkm = bkm.fit(df_features).transform(df_features)
        
        pred_bkm_filtered = pred_bkm.filter(col("topics").isNotNull() & (col("topics") != "NaN"))
        pred_bkm_filtered = pred_bkm_filtered.withColumn("topics_array", split(col("topics"), ";"))
        pred_bkm_exploded = pred_bkm_filtered.withColumn("topic", explode(col("topics_array")))
        
        purity_bkm, entropy_bkm = calculate_purity_entropy(pred_bkm_exploded)
        if purity_bkm is not None:
            print(f"Purity: {purity_bkm:.4f}")
            print(f"Entropy: {entropy_bkm:.4f}")
            results.append({
                "k": k,
                "method": "BisectingKMeans",
                "variant": variant,
                "purity": purity_bkm,
                "entropy": entropy_bkm
            })
spark.stop()
df_results = pd.DataFrame(results)
df_results.to_csv("../Data/csv/results.csv", index=False, mode="a", header=False)
print("✅ Đã lưu kết quả vào 'results.csv'")



============================== numFeatures=9820 ==============================


['id', 'topics', 'features']

============================== k=5 ==============================

KMeans


Purity: 0.5256
Entropy: 4.0193

BisectingKMeans


Purity: 0.4946
Entropy: 4.7230

============================== k=6 ==============================

KMeans


Purity: 0.5268
Entropy: 3.6821

BisectingKMeans


Purity: 0.4946
Entropy: 4.1645

============================== k=7 ==============================

KMeans


Purity: 0.5233
Entropy: 3.9494

BisectingKMeans


Purity: 0.5107
Entropy: 4.2806

============================== k=8 ==============================

KMeans


Purity: 0.5254
Entropy: 2.8705

BisectingKMeans


Purity: 0.5107
Entropy: 4.5036
✅ Đã lưu kết quả vào 'results.csv'
